### Extract Bonnet & Blowhead coordinates

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model, save_model
from sklearn.model_selection import train_test_split

### Load Train/Test Data

In [ ]:
def load_train_images_to_dataframe(folder_path):
    # Initialize an empty list to store image information
    image_data = []
    image_file_name = []

    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    counter = 0
    # Iterate over each file
    for file_name in files:
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file_name)
        
        # file_name_without_resized = file_name[8:]
        
        # To filter out the test images
        # if file_name_without_resized not in test_image_names.values:
        if file_name not in test_image_names.values:
            counter += 1
            # Read the image
            img = cv2.imread(file_path)
            
            # # Check if the image is loaded properly
            if img is None:
                print(f"Error: Image {file_name} not loaded!")
                continue
            image_file_name.append(file_name)
            image_data.append(img)

    # Create a DataFrame from the list of image information
    df = np.array(image_data)
    
    return image_file_name, df

def load_test_images_to_dataframe(folder_path):
    # Initialize an empty list to store image information
    image_data = []
    image_file_name = []
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    counter = 0
    # Iterate over each file
    for file_name in files:
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file_name)
        
        # file_name_without_resized = file_name[8:]
        
        # if file_name_without_resized in test_image_names.values:
        if file_name in test_image_names.values:
            counter += 1
            # Read the image
            img = cv2.imread(file_path)
            
            # Check if the image is loaded properly
            if img is None:
                print(f"Error: Image {file_name} not loaded!")
                continue
            
            image_data.append(img)
            image_file_name.append(file_name)
            
    # Create a DataFrame from the list of image information
    df = np.array(image_data)
    
    return image_file_name, df

In [ ]:
# Load the first CSV file with scaling ratios
scaling_data = pd.read_csv('image_info.csv')

# Load the second CSV file with image data
image_data = pd.read_csv('annotations/train_with_points.csv')

# Merge the two dataframes based on the 'Image' and 'file_name' columns
merged_data = pd.merge(image_data, scaling_data, left_on='Image', right_on='file_name')

# Apply scaling ratios to the coordinate columns
merged_data['bonnet_tip_x_rescaled'] = merged_data['bonnet_tip_x'] * merged_data['x_scale_ratio']
merged_data['bonnet_tip_y_rescaled'] = merged_data['bonnet_tip_y'] * merged_data['y_scale_ratio']
merged_data['blowhead_x_rescaled'] = merged_data['blowhead_x'] * merged_data['x_scale_ratio']
merged_data['blowhead_y_rescaled'] = merged_data['blowhead_y'] * merged_data['y_scale_ratio']

# Select and reorder the columns for the final CSV file
final_data = merged_data[['Image', 'whaleID', 'bonnet_tip_x_rescaled', 'bonnet_tip_y_rescaled', 'blowhead_x_rescaled', 'blowhead_y_rescaled']]

# Write the final data to a CSV file
final_data.to_csv('annotations/train_with_points_scaled.csv', index=False)

In [ ]:
def load_data(csv_file):
    df = pd.read_csv(csv_file)
    whale_ids = df['whaleID']
    bonnet_tip_coords = df[['bonnet_tip_x_rescaled', 'bonnet_tip_y_rescaled']]
    blowhead_coords = df[['blowhead_x_rescaled', 'blowhead_y_rescaled']]
    return whale_ids, bonnet_tip_coords, blowhead_coords

In [ ]:
image_folder = 'dataset/imgs256x256#2'
csv_file = 'annotations/train_with_points_scaled.csv'

test_image_names = pd.read_csv('dataset/sample_submission.csv')['Image']
images = load_train_images_to_dataframe(image_folder)
whale_ids, bonnet_tip_coords, blowhead_coords = load_data(csv_file)

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, pd.concat([bonnet_tip_coords, blowhead_coords], axis=1), test_size=0.2, random_state=42)

In [ ]:
# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, name='coordinates')  # Output four coordinates: bonnet_tip_x, bonnet_tip_y, blowhead_x, blowhead_y
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
print(model.summary())

c:\Users\Aniket Konkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ coordinates (Dense)             │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,492 (56.61 MB)

 Trainable params: 14,839,492 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=11, batch_size=32)

KeyboardInterrupt: 

In [ ]:
# Save model
save_model(model, "models/recognize_bonnet_blowhole#2.h5")

In [ ]:
x_test = load_test_images_to_dataframe('dataset/imgs256x256#2')
x_test.shape

(6925, 256, 256, 3)

In [ ]:
coordinates_pred = model.predict(x_test)

217/217 ━━━━━━━━━━━━━━━━━━━━ 26s 121ms/step


In [ ]:
coordinates_pred

array([[158.51926, 174.79486, 149.22911, 151.68988],
       [102.97779, 123.28926, 113.98486, 125.57421],
       [124.59868,  80.73863, 141.33603, 106.04277],
       ...,
       [ 79.60274, 149.23119,  92.75433, 140.03893],
       [135.57701,  71.28109, 123.9413 ,  62.22837],
       [162.52406, 201.4274 , 124.18299, 138.36237]], dtype=float32)

### Save the Bonnet&Blowhead prediction to a CSV

In [ ]:
bonnet_blowhead_coord_csv = pd.read_csv('dataset/sample_submission.csv')
bonnet_blowhead_coord_csv = bonnet_blowhead_coord_csv['Image']
coordinates_df = pd.DataFrame(coordinates_pred, columns=['bonnet_tip_x', 'bonnet_tip_y', 'blowhead_x', 'blowhead_y'])
bonnet_blowhead_coord_csv = pd.merge(bonnet_blowhead_coord_csv, coordinates_df, left_index=True, right_index=True)
bonnet_blowhead_coord_csv

,Image,bonnet_tip_x,bonnet_tip_y,blowhead_x,blowhead_y
0,w_1947.jpg,158.519257,174.794861,149.229111,151.689880
1,w_11096.jpg,102.977791,123.289261,113.984863,125.574211
2,w_10973.jpg,124.598679,80.738632,141.336029,106.042770
3,w_10442.jpg,122.838097,89.135063,117.294724,59.991535
4,w_10606.jpg,53.179543,73.136780,82.248489,100.379349
...,...,...,...,...,...
6920,w_4867.jpg,160.389038,177.345230,145.316177,152.102234
6921,w_5230.jpg,114.591728,119.105042,115.312263,108.523911
6922,w_5278.jpg,79.602737,149.231186,92.754333,140.038925
6923,w_9218.jpg,135.577011,71.281090,123.941299,62.228371


In [ ]:
bonnet_blowhead_coord_csv.to_csv('dataset/bonnet_blowhead_coordinates#2.csv', index=False)